In [1]:
import sqlite3
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as ptch

from search_analysis import search_analysis
import numpy as np
import pandas as pd
import validators
import math
from pyvis.network import Network

from bs4 import BeautifulSoup
import requests
from collections import deque
import time
import re

from Search import Search


import sys

# from Search import Search

%load_ext autoreload
%autoreload 2

d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
search_obj : Search = pd.read_pickle("very_small_test.pickle")

d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [ ]:
len(search_obj.cve_set)

In [ ]:
len(search_analysis.cve_sets["dec_2022_1"])

In [3]:
# con = sqlite3.connect("cve_july.db")
toy_set = search_analysis(pickled_search=search_obj)
# con.close()

d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [13]:
toy_set.term_url

,url,term,domain_url
0,https://en.wikipedia.org/wiki/Duck,duck,en.wikipedia.org
1,https://en.wikipedia.org/w/index.php?title=Duc...,duck,en.wikipedia.org


d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [ ]:
toy_set.pickle_data(folder_location='pickled_dfs')

In [ ]:
len(set(toy_set.cve_anal_df["cve"].to_list()))

In [ ]:
A= pd.Series(search_obj.edge_list).explode().reset_index()
# pd.Series(search_obj.cve_url_pairs).explode().index
A.rename(columns={"index" : "url", 0 : "link"}, inplace=True)

In [ ]:
toy_set.append_num_cve_per_url()

In [ ]:
toy_set.cve_anal_df

In [ ]:
toy_set.cve_url[toy_set.cve_url["cve"] == "2022-35914"]

In [ ]:
used = toy_set.cve_url
used.loc[used["cve"] == "2022-35914", "url"].drop_duplicates() 

### Testing Search Result overlap With Linked Urls, Currently SR > L_URL

In [ ]:
con = sqlite3.connect("cve_july.db")
all = pd.read_sql("SELECT DISTINCT url, cve FROM linked_urls ", con)

all = all.loc[all['cve'].isin(toy_set.analyzed_cve_set), "url"]

seeds = pd.read_sql("SELECT DISTINCT url, cve FROM search_results", con)
seeds = seeds.loc[seeds['cve'].isin(toy_set.analyzed_cve_set), "url"]
con.close()


In [ ]:
all = set(all)
seeds = set(seeds)
# #cves * #seeds * #rel link on average ~= #rel_searches
seeds

### Testing to ensure the found CVES are the same as the promised cves

In [ ]:
con = sqlite3.connect("cve_july.db")
linked_cves = pd.read_sql("SELECT DISTINCT cve FROM linked_urls", con)
con.close()

In [ ]:
linkcve = set(linked_cves['cve'])
deccve = set(toy_set.analyzed_cve_set)

print(f"linked urls cve set has len = {len(linkcve)}")
print(f"december cve set has len = {len(deccve)}")
print(f"their intersection has len = {len(deccve.intersection(linkcve))}")
print(f"their symmertric difference has len = {len(deccve.symmetric_difference(linkcve))}")

linkcve < deccve


In [ ]:
keys = list(search_analysis.cve_sets.keys())

dec_keys = []

for i in keys:
    if str.__contains__(i, "dec"):
        dec_keys.append(i)

dec_sets = [search_analysis.cve_sets[i] for i in keys]
dec_cve_set = set().union(*dec_sets)


jul_keys = []

for i in keys:
    if str.__contains__(i, "jul"):
        jul_keys.append(i)

jul_sets = [search_analysis.cve_sets[i] for i in keys]

jul_cve_set = set().union(*jul_sets)


In [ ]:
dec_cve_set.symmetric_difference(jul_cve_set)

## General Unit Testing, does not test for correct outputs just correct types and no crashes

In [ ]:
### BIG TEST ###
# assumes toy_set loaded

cve_set = toy_set.get_cve_set()
seeds = toy_set.seeds
domains = pd.unique(toy_set.get_domain_set())

print("---testing all cve related methods---")

print(cve_set.__contains__("source"))

counter = 0
for cve in cve_set:
    if counter % int(len(cve_set)/100) == 0:
        print(f"{str(100 * counter/len(cve_set))[0:5]}%")
    counter += 1   
    assert isinstance(toy_set.get_cvss_score(cve, version='3'), float), "get cvss broken"

    assert isinstance(toy_set.resource_per_cve(cve), int), "resource_per_cve default broken"
    assert isinstance(toy_set.resource_per_cve(cve, return_list=True), pd.Series), "resource_per_cve list broken"
    assert isinstance(toy_set.resource_per_cve(cve, unique_domain=True), int), "resource_per_cve unique broken"
    assert isinstance(toy_set.resource_per_cve(cve, return_list=True, unique_domain=True), pd.Series), "resource_per_cve unique, list broken"

    assert isinstance(toy_set.num_connected_components_by_cve(cve), int), "num_connected_components_by_cve default broken"
    assert isinstance(toy_set.num_connected_components_by_cve(cve, by_domain=True), int), "num_connected_components_by_cve domain broken"

    assert isinstance(toy_set.get_reciprocity(cve), float), f"get_reciprocity default broken at {cve}"
    assert isinstance(toy_set.get_reciprocity(cve, by_domain=True), float), "get_reciprocity domain broken"

    assert isinstance(toy_set.get_restricted_edge_set(cve), tuple), "get_restricted_edge_set default broken"
    assert isinstance(toy_set.get_restricted_edge_set(cve, by_domain=True), tuple), "get_restricted_edge_set domain broken"
        





In [ ]:
print("---testing all seed related methods---")
counter = 0
for seed in seeds:
    if counter % int(len(seeds)/10) == 0:
        print(f"{str(100 * counter/len(seeds))[0:5]}%")
    counter += 1

    assert isinstance(toy_set.get_cve_per_seed(domain=seed), int), "get cve per seed broken"
    assert isinstance(toy_set.get_cve_per_seed(domain=seed, return_cve_set=True), set), f"get cve per seed (set) broken at {seed}, {toy_set.get_cve_per_seed(domain=seed, return_cve_set=True)}"



In [ ]:
print("---testing all domain related methods---")
for domain in domains:
    assert isinstance(toy_set.get_cves_at_domain(domain), np.ndarray), "get_cves_at_domain default broken"
    assert isinstance(toy_set.get_cves_at_domain(domain, return_list=False), int), "get_cves_at_domain int broken"

    assert isinstance(toy_set.get_url_by_domain(domain), pd.Series), "get_url_by_domain broken" #currently not testing other edge lists



print("---testing all misc methods---")

try:
    toy_set.get_num_seeds_per_cve()
except:
    print("get_num_seeds_per_cve broken")

try:
    toy_set.get_cve_per_domain()
except:
    print("get_cve_per_domain broken")

try:
    toy_set.gen_correlation_matrix()
except:
    print("gen_correlation_matrix broken")

In [ ]:
toy_set.append_cve_per_seed()

toy_set.append_resource_per_cve()

toy_set.append_resource_per_cve(by_domain=True, reappend=True)

toy_set.append_ee_score()

toy_set.append_cvss_score()

toy_set.append_num_cve_per_url()

toy_set.append_num_connected_components()

toy_set.append_num_connected_components(by_domain=True, reappend=True)


### General overlap testing for various parts of the data set

In [ ]:
con = sqlite3.connect("cve_partial.db")
all_searches = pd.read_sql("SELECT DISTINCT url FROM search_results", con=con)
all_searches = all_searches["url"].to_list()

# searches = pd.read_sql("SELECT DISTINCT url FROM search_results WHERE url IN (SELECT DISTINCT url FROM linked_urls) ", con=con)
urls = pd.read_sql(sql="SELECT DISTINCT url FROM linked_urls", con=con)
urls = urls["url"].to_list()


con.close()

In [ ]:
# print(f"num of relevent searches: {all_searches.drop_duplicates()}")
print(f"num of distinct urls in search_results: {len(set(all_searches))}")
# print(all_searches)

print(f"num distinct urls in linked_urls: {len(set(urls))}")
# print(f"num source nodes in graph set: {len(set(urls.shape))}")


print(f"symmetric differnece between the two: {len(set(all_searches).symmetric_difference(set(urls)))}")
print(f"intersection between the two: {len(set(all_searches).intersection(set(urls)))}")
print(f"search results superset of urls?: {set(all_searches) > set(urls)}")



# print(all_searches.merge(right=urls, how="inner", on="url").shape)


In [ ]:
con = sqlite3.connect(database="cve_july.db")

seeds = pd.read_sql("SELECT DISTINCT url, cve FROM search_results", con)
seeds = seeds.loc[seeds['cve'].isin(toy_set.analyzed_cve_set), "url"]
con.close()

In [ ]:
# set(toy_set.cve_df[toy_set.cve_df["cve"] == "2014-0160"]["url"]) - set(toy_set.get_restricted_edge_set("2014-0160")["url"])

cve_df_urls = set(pd.unique(toy_set.cve_url["url"]))
edge_list_urls = set(pd.unique(toy_set.edge_list["url"]))
edge_list_links = set(pd.unique(toy_set.edge_list["link"]))

edge_list_nodes = set.union(edge_list_urls, edge_list_links)

all_nodes = set.union(edge_list_urls, cve_df_urls, edge_list_links)

seed_set = set(seeds.to_list())


print(f"len of cve_df_urls: {len(cve_df_urls)}")
print(f"len of edge_list_urls: {len(edge_list_urls)}")
print(f"len of edge_list_links: {len(edge_list_links)}")
print(f"len of all_nodes(edge_list_urls + edge_list_links + cve_df_urls): {len(all_nodes)}")
print(f"len of cve_df_urls - edge_list_nodes: {len(cve_df_urls - edge_list_nodes)}")
print(f"len of edge_list_nodes - cve_df_urls: {len(edge_list_nodes - cve_df_urls)}")

print(f"difference of all nodes and search results nodes should not be zero{len(all_nodes - seed_set)}")





'''# set(toy_set.seeds) > set(pd.unique(toy_set.cve_df[~toy_set.cve_df["url"].isin(all_nodes)]["domain_url"]))

# edges_start = set(pd.unique(edges["url"]))
# stunted = set(pd.unique(stunted_nodes["url"]))
# print(f"len of edges_start: {len(edges_start)}")
# print(f"len of stunted: {len(stunted)}")
# print(f"len of edges_start union stunted: {len(edges_start.union(stunted))}")

# edges_start < stunted # True so stunted is the superset, which makes sense

# stunted == cve_df_urls # True which also makes sense





# toy_set.edge_list[toy_set.edge_list["url"] == "https://heartbleed.com/"]''';


seeds[seeds.str.contains("woody")]

### Build the big graph

In [12]:
g = toy_set.build_graph_from_edgelist(toy_set.edge_list, source="url", target="link", remove_self_loop=True, ignore_root=True)
# g = nx.from_pandas_edgelist(df=toy_set.edge_list, source="domain_url", target="domain_link", create_using=nx.DiGraph)
net = Network(directed=True, select_menu=True, filter_menu=True,neighborhood_highlight=True)
net.from_nx(g)
net.show("torus.html")

d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [ ]:
test = nx.to_pandas_adjacency(g, dtype=bool)

### Various Graph Generations

In [ ]:
toy_set.append_cvss_score()
toy_set.append_resource_per_cve(reappend=True)

data = toy_set.cve_anal_df[["cve", "cvss_score", "resource_per_cve"]].dropna()
fig, ax = plt.subplots()

ax.set_xlabel("cvss_score")
ax.set_ylabel("# resources")
ax.set_yscale("log")

ax.scatter(y = data["resource_per_cve"], x = data["cvss_score"])


In [ ]:
toy_set.append_cvss_score()
toy_set.append_ee_score()

data = toy_set.cve_anal_df[["cve", "cvss_score", "ee_score"]].dropna()
fig, ax = plt.subplots()

ax.set_ylabel("cvss_score")
ax.set_xlabel("ee_score")
ax.scatter(x = data["ee_score"], y = data["cvss_score"])

In [ ]:
plt.hist(toy_set.cve_anal_df["resource_per_cve"])
plt.yscale("log")

In [ ]:
toy_set.gen_cvss_vs_resource_per_cve()


In [ ]:
fig, ax = toy_set.gen_cve_per_domain(stacked=True)

ax.set_yscale("log")

toy_set.cve_url[toy_set.cve_url["domain_url"] == 'zigrin.com']

In [ ]:
min(toy_set.get_cve_per_domain(unique=False)['cve'])

In [ ]:
fig, ax = toy_set.gen_cve_per_seeds()
ax.set_yscale("log")

In [ ]:
toy_set.seed_anal_df
# toy_set.cve_url[toy_set.cve_url["domain_url"].str.contains("dhs")]

In [ ]:
#Correlation matrix
a = toy_set.gen_correlation_matrix()

In [ ]:
#Show corelation matrix
g : nx.DiGraph = nx.from_pandas_adjacency(df = a, create_using=nx.DiGraph)

net = Network(directed=True, bgcolor="#222222", font_color="white", select_menu=True)
net.force_atlas_2based()
net.from_nx(g, edge_weight_transf=lambda x : x, edge_scaling=True)
net.show_buttons(filter_=['physics'])
net.show("correlation_matrix.html")

In [ ]:
#centrality testing
conectivity_test = nx.from_pandas_edgelist(toy_set.edge_list, source="domain_url", target="domain_link", create_using=nx.DiGraph)
b = nx.closeness_centrality(conectivity_test)


domains["connectivity"] = domains["domain_url"].map(b)
domains["cve_per_domain"] = domains["domain_url"].map(lambda x : toy_set.get_cves_at_domain(domain=x, return_list=False))

In [ ]:
#centrality testing
plt.hist(x=domains["connectivity"])
plt.ylabel("connectivity")

plt.yscale("log")


In [ ]:
#centrality testing
cves = toy_set.get_cve_set()

g : nx.DiGraph = nx.from_pandas_edgelist(toy_set.edge_list, source="domain_url", target="domain_link", create_using=nx.DiGraph)

net = Network(directed=True, bgcolor="#3d3d3d", font_color="white")
net.from_nx(g)
net.force_atlas_2based()



for node in net.nodes:
    node["color"] = "#04f2b2" if node["id"] in list(domains.loc[domains["connectivity"] < 0.075, "domain_url"]) else "#04baf2"

net.show("connectivity.html")

In [ ]:
toy_set.append_num_seeds_per_cve(reappend=True)

In [ ]:
plt.scatter(toy_set.cve_anal_df['num_seeds_per_cve'], toy_set.cve_anal_df['num_connected_components'])
plt.xlabel('# seeds per cve')
plt.ylabel('# connected components')
# plt.yscale('log')

plt.title('CVSS Versus Number of Seeds')



### Wierd Metric

In [70]:
def num_con_d(x:nx.Graph, y:nx.Graph):
    # x_undir = nx.to_undirected(x)
    num_cons_x : int = nx.number_connected_components(x)
    # y_undir = nx.to_undirected(y)
    num_cons_y : int = nx.number_connected_components(y)
    union = nx.compose(x, y)
    num_cons_union = nx.number_connected_components(union)
    return num_cons_union / (num_cons_x + num_cons_y)

def num_con_d_pre_calc(x:tuple, y:tuple):
    union = nx.compose(x[0], y[0])
    num_cons_union = nx.number_connected_components(union)
    return num_cons_union / (x[1] + y[1])




In [ ]:
cve_toy = list(toy_set.get_cve_set())
num_cves = len(cve_toy) 

In [ ]:
restricted_edges = dict()

for cve in cve_toy:
    edges, nodes = toy_set.get_restricted_edge_set(cve, by_domain=False)
    g : nx.Graph = nx.from_pandas_edgelist(edges, source="url", target="link", create_using=nx.Graph)
    g.add_nodes_from(nodes)

    restricted_edges[cve] = (g, nx.number_connected_components(g))

In [ ]:
cve_adj = pd.DataFrame(data=np.zeros((num_cves,num_cves)), index=cve_toy, columns=cve_toy)



count = 0
for i in cve_toy:
    if count % 10 == 0:
        print(f'{str(100 * count / num_cves)[0:5]}% done, {num_cves - count} remaining')
    count += 1
    
    x :nx.Graph = restricted_edges[i]
    if len(x[0].nodes) == 0:
        continue
    for j in cve_toy:
        if j >= i:
            continue
        y = restricted_edges[j]

        if len(y[0].nodes) == 0:
            continue

        d_x_y = num_con_d_pre_calc(x, y)

        if d_x_y > 0.6:
             continue
        
        cve_adj.at[i,j] = d_x_y
        cve_adj.at[j,i] = d_x_y






In [ ]:
np.unique(np.array(cve_adj).flatten())


In [ ]:
g : nx.Graph = nx.from_pandas_adjacency(cve_adj)
g.remove_nodes_from(list(nx.isolates(g)))

net = Network(directed=False, bgcolor="#222222", font_color="white")
net.from_nx(g,  edge_scaling=False)
net.force_atlas_2based()
net.show("cve_distances.html")

### Get Size of SA

In [ ]:
data = []
for i in toy_set.__dict__.keys():
    print(i)
    data.append(i.__sizeof__())

print(np.sum(data))
print(pd.unique(toy_set.cve_anal_df.drop_duplicates()["ee_score"]))

### Get Set of Search Results

In [ ]:
a = a[a['url'].isin(toy_set.url_found_in_search_set["url"])]


In [ ]:
set(toy_set.url_found_in_search_set["url"]) == set(toy_set.cve_url["url"])

In [ ]:
a = toy_set.url_found_in_search_set[toy_set.url_found_in_search_set["cve"] == '2022-27660']
a

In [ ]:
for i in list(a['url']):
    print(i + ',')

len(a)

In [ ]:
session = requests.Session()
headers = {
    "User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"
}
con = sqlite3.connect("cve_july.db")

cves = ['2022-4731', '2022-3273', '2022-22477', '2022-3272', '2022-29465', '2022-2550']

blocked_url_prefixes = {'packetstormsecurity.com/files/date',
                        'packetstormsecurity.com/files/author/',
                        'packetstormsecurity.com/files/download/',
                        'isc.sans.edu/handler_list/',
                        'cassandra.cerias.purdue.edu/CVE_changes',  # contains db of all CVE changes by month
                        'cve.mitre.org/cgi-bin/cvekey.cgi?keyword=',
                        'cve.mitre.org/data/refs/',
                        'nvd.nist.gov/vuln/full-listing/',
                        'nvd.nist.gov/vuln/search/results',
                        'krebsonsecurity.com/tag/',
                        'www.askwoody.com/forums/users/',
                        'www.askwoody.com/author/'}

document = open("new_cves.csv", mode='w')

for cve in cves:
    a = pd.read_sql(f"SELECT DISTINCT url FROM search_results WHERE cve = '{cve}'", con)

    print(len(list(a['url'])))

    for url in list(a['url']):
        should_be_blocked = False
        for blocked in blocked_url_prefixes:
            if str.__contains__(str.lower(url), str.lower(blocked)):
                print('blocked: ' + url)
                should_be_blocked = True
                break
        if should_be_blocked:
            continue
        
        try:
            request = session.get(url=url, headers=headers)
        except:
            print("url failed: " + cve + ',' + url)
            continue

        soup = BeautifulSoup(request.content, 'html.parser')
        soup_text = str.lower(soup.text)
        if cve in soup_text:
            document.write(cve + ',' + url + ',\n')
            
        

con.close()

document.close()


    

In [ ]:
session = requests.Session()
headers = {
    "User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"
}

url = 'https://www.cisa.gov/news-events/bulletins/sb22-206'

request = session.get(url=url, headers=headers)

soup = BeautifulSoup(request.content, 'html.parser')


### Reconstructing the man

In [ ]:
high_cvss_high_count = set(['2022-4687', '2022-2493', '2022-2550', '2022-2829', '2022-4809', '2022-4800', '2022-3037', '2022-4767', '2022-3072', '2022-2652', '2022-4804', '2022-3520', '2022-2631', '2022-4802', '2022-4136', '2022-26346', '2022-4813', '2022-20842', '2022-2344', '2022-2626'])
high_cvss_low_count = set(['2022-27660', '2022-41794', '2022-29465', '2022-22489', '2022-41649', '2022-24017', '2022-32774', '2022-35875', '2022-30690', '2022-20822', '2022-40649', '2022-35878', '2022-40644', '2022-34652', '2022-29468', '2022-28677', '2022-43599', '2022-32282', '2022-38097', '2022-40645'])
med_cvss_high_count = set(['2022-20906', '2022-3326', '2022-3272', '2017-20151', '2022-2965', '2022-3869', '2022-3456', '2022-2353', '2022-22490', '2022-4414', '2021-4276', '2022-3002', '2022-4725', '2022-3364', '2022-4721', '2022-20800', '2022-3000', '2022-4719', '2022-4846', '2022-4810'])
med_cvss_low_count =  set(['2022-20899', '2021-29790', '2022-22477', '2021-38936', '2022-35280', '2018-25053', '2022-36774', '2022-22373', '2022-20810', '2022-30613', '2022-43588', '2022-35722', '2022-35715', '2022-20880', '2022-43595', '2022-20882', '2022-20878', '2022-31774', '2022-20883', '2022-34162'])
low_cvss_high_count =  set(['2022-2822', '2021-4283', '2022-3273', '2022-4617', '2022-3363', '2022-3457', '2022-4736', '2021-4277', '2022-4755', '2022-4820', '2022-4632', '2021-4285', '2017-20153', '2022-4823', '2022-4069', '2022-4614', '2022-4859', '2017-20155', '2022-4560', '2019-25088'])
low_cvss_low_count =  set(['2022-22450', '2022-34874', '2022-4731', '2018-25054', '2022-28681', '2019-25091', '2018-25059', '2022-35286', '2020-36638', '2020-36634', '2021-39041', '2018-25060', '2020-36635', '2022-22455', '2020-36636', '2018-25051', '2022-28670', '2022-22460', '2022-44744', '2018-25055'])

union = set.union(high_cvss_high_count, high_cvss_low_count, med_cvss_high_count, med_cvss_low_count, low_cvss_high_count, low_cvss_low_count)

c = []

for i in union:
    if not toy_set.analyzed_cve_set.__contains__(i):
        c.append(i)
        

print(len(union))